# Fast-Fourier-Transformation Example

## Machine-Learned Inverse FFT

Let's start with a sinusoidal curve: that's the input (`x`) that we want to learn, given its FFT (`y`).
The goal is to do an `InverseRealFFT` by gradient descent.

For this problem the input (`x`) is real (`float32`) and label (`y`), the FFT, is complex (`complex64`).

In [1]:
!*rm -f go.work && go work init
!*go work use . "/opt/janpf/Projects/gomlx.gomlx"
%goworkfix

	- added replace rule for module "github.com/gomlx/gomlx" to local directory "/opt/janpf/Projects/gomlx.gomlx".

In [78]:
import (
    . "github.com/gomlx/gomlx/graph"
    . "github.com/gomlx/gomlx/types/exceptions"
    mg "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/janpfeifer/gonb/gonbui"
)

// manager always created at initialization.
var manager = NewManager()

const (
    NumPoints = 100
    Frequency = 2.0  // Number of curves in samples.
    RealDType = shapes.Float32
    ComplexDType = shapes.Complex64
)

// CalculateXY returns (x, y) of our problem, where y is a sinusoidal curve and x is its FFT.
func CalculateXY() (x, y tensor.Tensor) {
    e := NewExec(manager, func (g *Graph) (x, y *Node) {
        x = Iota(g, shapes.Make(RealDType, 1, NumPoints), 1)
        x = MulScalar(x, 2.0*math.Pi*Frequency/float64(NumPoints))
        x = Sin(x)
        y = RealFFT(x)
        return
    })
    res := e.Call()
    return res[0], res[1]
}

func Plot(displayId string, width, height int, tensors ...tensor.Tensor) {
    plts := mg.New(width, height)
    for ii, t := range tensors {
        var values []float64
        switch t.DType() {
        case shapes.F64:
            values = t.Local().Flat().([]float64)
        case shapes.F32:
            values32 := t.Local().Flat().([]float32)
            values = slices.Map(values32, func (v float32) float64 { return float64(v) })
        default:
            Panicf("only float32 and float64 tensor dtypes are accepted by Plot, got t.shape=%s", t.Shape())
        }
        var name string
        if len(tensors) > 0 {
            name = fmt.Sprintf("#%d", ii)
        }
        plts.AddValues(name, "", values)
    }
    if displayId == "" {
        plts.Plot()
    } else {
        gonbui.UpdateHTML(displayId, plts.PlotToHTML())
    }
}

%%
x, y := CalculateXY()
fmt.Printf("x: shape=%s\n", x.Shape())
fmt.Printf("y: shape=%s\n", y.Shape())
Plot("", 1024, 320, x)

x: shape=(Float32)[1 100]
y: shape=(Complex64)[1 51]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,9.000000e+01 L2.625212e+01,1.008502e+02 L3.482424e+01,1.115293e+02 L4.339636e+01,1.218689e+02 L5.196848e+01,1.317058e+02 L6.054061e+01,1.408851e+02 L6.911273e+01,1.492618e+02 L7.768485e+01,1.567040e+02 L8.625697e+01,1.630942e+02 L9.482909e+01,1.683316e+02 L1.034012e+02,1.723338e+02 L1.119733e+02,1.750374e+02 L1.205455e+02,1.764000e+02 L1.291176e+02,1.764000e+02 L1.376897e+02,1.750374e+02 L1.462618e+02,1.723337e+02 L1.548339e+02,1.683316e+02 L1.634061e+02,1.630942e+02 L1.719782e+02,1.567040e+02 L1.805503e+02,1.492618e+02 L1.891224e+02,1.408851e+02 L1.976945e+02,1.317058e+02 L2.062667e+02,1.218688e+02 L2.148388e+02,1.115293e+02 L2.234109e+02,1.008502e+02 L2.319830e+02,9.000000e+01 L2.405552e+02,7.914979e+01 L2.491273e+02,6.847070e+01 L2.576994e+02,5.813114e+01 L2.662715e+02,4.829417e+01 L2.748436e+02,3.911494e+01 L2.834158e+02,3.073819e+01 L2.919879e+02,2.329603e+01 L3.005600e+02,1.690582e+01 L3.091321e+02,1.166837e+01 L3.177042e+02,7.666243e+00 L3.262764e+02,4.962576e+00 L3.348485e+02,3.600005e+00 L3.434206e+02,3.600005e+00 L3.519927e+02,4.962587e+00 L3.605648e+02,7.666258e+00 L3.691370e+02,1.166838e+01 L3.777091e+02,1.690585e+01 L3.862812e+02,2.329605e+01 L3.948533e+02,3.073821e+01 L4.034255e+02,3.911497e+01 L4.119976e+02,4.829422e+01 L4.205697e+02,5.813120e+01 L4.291418e+02,6.847076e+01 L4.377139e+02,7.914981e+01 L4.462861e+02,9.000002e+01 L4.548582e+02,1.008502e+02 L4.634303e+02,1.115293e+02 L4.720024e+02,1.218689e+02 L4.805745e+02,1.317058e+02 L4.891467e+02,1.408851e+02 L4.977188e+02,1.492618e+02 L5.062909e+02,1.567040e+02 L5.148630e+02,1.630942e+02 L5.234352e+02,1.683316e+02 L5.320073e+02,1.723338e+02 L5.405794e+02,1.750374e+02 L5.491515e+02,1.764000e+02 L5.577236e+02,1.764000e+02 L5.662958e+02,1.750374e+02 L5.748679e+02,1.723337e+02 L5.834400e+02,1.683316e+02 L5.920121e+02,1.630941e+02 L6.005842e+02,1.567039e+02 L6.091564e+02,1.492618e+02 L6.177285e+02,1.408850e+02 L6.263006e+02,1.317058e+02 L6.348727e+02,1.218688e+02 L6.434448e+02,1.115292e+02 L6.520170e+02,1.008501e+02 L6.605891e+02,9.000000e+01 L6.691612e+02,7.914979e+01 L6.777333e+02,6.847070e+01 L6.863055e+02,5.813114e+01 L6.948776e+02,4.829417e+01 L7.034497e+02,3.911493e+01 L7.120218e+02,3.073817e+01 L7.205939e+02,2.329602e+01 L7.291661e+02,1.690582e+01 L7.377382e+02,1.166836e+01 L7.463103e+02,7.666243e+00 L7.548824e+02,4.962576e+00 L7.634545e+02,3.600000e+00 L7.720267e+02,3.600010e+00 L7.805988e+02,4.962597e+00 L7.891709e+02,7.666274e+00 L7.977430e+02,1.166841e+01 L8.063152e+02,1.690588e+01 L8.148873e+02,2.329609e+01 L8.234594e+02,3.073825e+01 L8.320315e+02,3.911502e+01 L8.406036e+02,4.829427e+01 L8.491758e+02,5.813117e+01 L8.577479e+02,6.847074e+01 L8.663200e+02,7.914983e+01 "/> 0 10 20 30 40 50 60 70 80 90 Steps -0.000 #0

In [77]:
%rm TrainInverseRealFFT

. removed func TrainInverseRealFFT


### Train the model

If you run it, you'll see the plot of the "learnedX" adjusting towards "x", the original sinusoidal curve.

In [89]:
import (
    . "github.com/gomlx/gomlx/graph"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/data"
    "github.com/gomlx/gomlx/ml/train"
)

var (
	flagNumSteps     = flag.Int("steps", 1000, "Number of gradient descent steps to perform")
	flagLearningRate = flag.Float64("learning_rate", 0.1, "Initial learning rate.")
)

func TrainInverseRealFFT() {
    x, y := CalculateXY()
    ctx := context.NewContext(manager)
	ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    modelFn := func(ctx *context.Context, spec any, inputs []*Node) []*Node {
        g := inputs[0].Graph()
        learnedXVar := ctx.VariableWithShape("learnedX", x.Shape())
        predictedY := RealFFT(learnedXVar.ValueGraph(g))
        return []*Node{predictedY}
    }

    dataset, err := data.InMemoryFromData(manager, "dataset", []any{x}, []any{y})
    if err != nil {
        panic(err)
    }
    dataset.BatchSize(1, false).Infinite(true)

    opt := optimizers.Adam().Done()
    trainer := train.NewTrainer(
        manager, ctx, modelFn,
        losses.MeanAbsoluteError,
        opt,
        nil, nil) // trainMetrics, evalMetrics

	loop := train.NewLoop(trainer)
	commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Plot learnedX
    displayId := gonbui.UniqueID()
    gonbui.UpdateHTML(displayId, "")
    train.EveryNSteps(loop, 10, "plot", 0, func(loop *train.Loop, metrics []tensor.Tensor) error {
        learnedXVar := ctx.InspectVariable(context.RootScope, "learnedX")
        learnedX := learnedXVar.Value()
        Plot(displayId, 1024, 320, x, learnedX)
        return nil
    })
    
	// Loop for given number of steps.
	_, err = loop.RunSteps(dataset, *flagNumSteps)
	if err != nil {
		panic(err)
	}

}

%% --steps=1000 --learning_rate=0.01
%env GOMLX_PLATFORM Host
fmt.Println(manager.Platform())
TrainInverseRealFFT()

Set: GOMLX_PLATFORM="Host"
Host


2023-08-05 10:55:47.041657: E external/xla/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


<path d="M1.768000e+01,9.000392e+01 L2.625212e+01,1.008424e+02 L3.482424e+01,1.115099e+02 L4.339636e+01,1.218383e+02 L5.196848e+01,1.316646e+02 L6.054061e+01,1.408339e+02 L6.911273e+01,1.492016e+02 L7.768485e+01,1.566357e+02 L8.625697e+01,1.630189e+02 L9.482909e+01,1.682507e+02 L1.034012e+02,1.722485e+02 L1.119733e+02,1.749493e+02 L1.205455e+02,1.763104e+02 L1.291176e+02,1.763104e+02 L1.376897e+02,1.749493e+02 L1.462618e+02,1.722485e+02 L1.548339e+02,1.682507e+02 L1.634061e+02,1.630189e+02 L1.719782e+02,1.566357e+02 L1.805503e+02,1.492016e+02 L1.891224e+02,1.408339e+02 L1.976945e+02,1.316646e+02 L2.062667e+02,1.218382e+02 L2.148388e+02,1.115099e+02 L2.234109e+02,1.008424e+02 L2.319830e+02,9.000391e+01 L2.405552e+02,7.916545e+01 L2.491273e+02,6.849793e+01 L2.576994e+02,5.816956e+01 L2.662715e+02,4.834324e+01 L2.748436e+02,3.917395e+01 L2.834158e+02,3.080626e+01 L2.919879e+02,2.337217e+01 L3.005600e+02,1.698889e+01 L3.091321e+02,1.175710e+01 L3.177042e+02,7.759311e+00 L3.262764e+02,5.058566e+00 L3.348485e+02,3.697470e+00 L3.434206e+02,3.697470e+00 L3.519927e+02,5.058577e+00 L3.605648e+02,7.759321e+00 L3.691370e+02,1.175711e+01 L3.777091e+02,1.698891e+01 L3.862812e+02,2.337219e+01 L3.948533e+02,3.080630e+01 L4.034255e+02,3.917399e+01 L4.119976e+02,4.834330e+01 L4.205697e+02,5.816962e+01 L4.291418e+02,6.849799e+01 L4.377139e+02,7.916548e+01 L4.462861e+02,9.000394e+01 L4.548582e+02,1.008424e+02 L4.634303e+02,1.115099e+02 L4.720024e+02,1.218383e+02 L4.805745e+02,1.316646e+02 L4.891467e+02,1.408339e+02 L4.977188e+02,1.492016e+02 L5.062909e+02,1.566357e+02 L5.148630e+02,1.630190e+02 L5.234352e+02,1.682507e+02 L5.320073e+02,1.722485e+02 L5.405794e+02,1.749493e+02 L5.491515e+02,1.763104e+02 L5.577236e+02,1.763104e+02 L5.662958e+02,1.749493e+02 L5.748679e+02,1.722485e+02 L5.834400e+02,1.682507e+02 L5.920121e+02,1.630189e+02 L6.005842e+02,1.566356e+02 L6.091564e+02,1.492015e+02 L6.177285e+02,1.408338e+02 L6.263006e+02,1.316645e+02 L6.348727e+02,1.218382e+02 L6.434448e+02,1.115098e+02 L6.520170e+02,1.008423e+02 L6.605891e+02,9.000392e+01 L6.691612e+02,7.916546e+01 L6.777333e+02,6.849793e+01 L6.863055e+02,5.816956e+01 L6.948776e+02,4.834324e+01 L7.034497e+02,3.917394e+01 L7.120218e+02,3.080625e+01 L7.205939e+02,2.337216e+01 L7.291661e+02,1.698888e+01 L7.377382e+02,1.175709e+01 L7.463103e+02,7.759305e+00 L7.548824e+02,5.058566e+00 L7.634545e+02,3.697465e+00 L7.720267e+02,3.697476e+00 L7.805988e+02,5.058587e+00 L7.891709e+02,7.759342e+00 L7.977430e+02,1.175714e+01 L8.063152e+02,1.698894e+01 L8.148873e+02,2.337223e+01 L8.234594e+02,3.080634e+01 L8.320315e+02,3.917403e+01 L8.406036e+02,4.834335e+01 L8.491758e+02,5.816960e+01 L8.577479e+02,6.849796e+01 L8.663200e+02,7.916549e+01 " vector-effect="non-scaling-stroke"/> <path d="M1.768000e+01,9.004325e+01 L2.625212e+01,1.007718e+02 L3.482424e+01,1.114816e+02 L4.339636e+01,1.217399e+02 L5.196848e+01,1.316801e+02 L6.054061e+01,1.407556e+02 L6.911273e+01,1.492350e+02 L7.768485e+01,1.566847e+02 L8.625697e+01,1.630623e+02 L9.482909e+01,1.681586e+02 L1.034012e+02,1.722993e+02 L1.119733e+02,1.749371e+02 L1.205455e+02,1.764000e+02 L1.291176e+02,1.762123e+02 L1.376897e+02,1.749303e+02 L1.462618e+02,1.722425e+02 L1.548339e+02,1.684117e+02 L1.634061e+02,1.630637e+02 L1.719782e+02,1.567383e+02 L1.805503e+02,1.492422e+02 L1.891224e+02,1.406948e+02 L1.976945e+02,1.317919e+02 L2.062667e+02,1.218485e+02 L2.148388e+02,1.115783e+02 L2.234109e+02,1.006641e+02 L2.319830e+02,8.989466e+01 L2.405552e+02,7.920999e+01 L2.491273e+02,6.842556e+01 L2.576994e+02,5.829832e+01 L2.662715e+02,4.827718e+01 L2.748436e+02,3.928394e+01 L2.834158e+02,3.081727e+01 L2.919879e+02,2.325405e+01 L3.005600e+02,1.689106e+01 L3.091321e+02,1.187508e+01 L3.177042e+02,7.682004e+00 L3.262764e+02,5.041546e+00 L3.348485e+02,3.765787e+00 L3.434206e+02,3.802131e+00 L3.519927e+02,5.189814e+00 L3.605648e+02,7.783938e+00 L3.691370e+02,1.164692e+01 L3.777091e+02,1.711683e+01 L3.862812e+02,2.339704e+01 L3.948533e+02,3.075969e+01 L4.034255e+02,3.905345e+01 L

Training (1000 steps):  100% [========================================] (4336 steps/s) [loss=0.012] [~loss=0.012]        
